<a href="https://colab.research.google.com/github/VincentGariepy/IFT6390-Competition2-BeaudelAIre/blob/main/Kernel_SVM/Comp%C3%A9tition_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Compétition 2
Anshita Saxena, Denis Lemarchand ()

#Code

In [ ]:
import numpy as np
import pandas as pd

##Upload Kaggle File from Google Drive

In [ ]:
#@title Install PyDrive
!pip install PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#This notebook should access to Google Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# train data
# https://drive.google.com/file/d/17cD6hFS_AfKxtfjKxfeNUkC44jfSYkGj/view?usp=sharing
# train result
# https://drive.google.com/file/d/1iEpwQ3B4d4gIUl3ynq_p74U1QKZTURem/view?usp=sharing
# test data
# https://drive.google.com/file/d/1QbhqakgLpBXWr5sMFSe-BQpFw_X2XZZR/view?usp=sharing

#download Kaggle Files from Google Drive to store it in Colab Session
downloaded = drive.CreateFile({'id':"17cD6hFS_AfKxtfjKxfeNUkC44jfSYkGj"})   
downloaded.GetContentFile('train.csv') 

downloaded = drive.CreateFile({'id':"1iEpwQ3B4d4gIUl3ynq_p74U1QKZTURem"})   
downloaded.GetContentFile('train_result.csv')    

downloaded = drive.CreateFile({'id':"1QbhqakgLpBXWr5sMFSe-BQpFw_X2XZZR"})   
downloaded.GetContentFile('test.csv')     

In [ ]:
#
df_train = pd.read_csv('train.csv')
train = df_train.to_numpy()
train_inputs = train[:]

#
df_train_result = pd.read_csv('train_result.csv')
train_results = df_train_result.to_numpy()
train_labels = train_results[:,1]

#
df_test = pd.read_csv('test.csv')
test = df_test.to_numpy()
test_inputs = test[:]

In [ ]:
train_labels[train_labels=='negative']=0
train_labels[train_labels=='neutral']=1
train_labels[train_labels=='positive']=2

In [ ]:
# Smoke Tests
list_classes = np.unique(train_labels)
print(list_classes)

n = np.random.randint(0, len(train_inputs))
print(train_inputs[n], train_labels[n])

n = np.random.randint(0, len(test_inputs))
print(test_inputs[n])

dist_train_labels = [np.sum(train_labels==0)/len(train_labels),
  np.sum(train_labels==1)/len(train_labels),
  np.sum(train_labels==2)/len(train_labels)]

print(dist_train_labels)

[0 1 2]
[622291
 'Watching Lake Placid 2, i loved the first one. hope i like this one toooo! '] 2
[246351 '@David_Mooney hi! Barely fuctioning! Crap day ']
[0.49965539548774757, 8.074415349848076e-05, 0.5002638603587539]


##Generate submission file

In [ ]:
#make submission file from numpy data
#The value of class should be 0, 1 or 2 with 0 being negative, 1 being neutral and 2 being positive class.
def PrepareKaggleFile(test_inputs, test_predictions, file='tests_label.csv'):
    output_data_for_kaggle = np.zeros((len(test_inputs),2))
    for i in range(len(test_inputs)):
      output_data_for_kaggle[i,0] = i
      output_data_for_kaggle[i,1] = test_predictions[i]

    output_data_for_kaggle = output_data_for_kaggle.astype(int)

    print(output_data_for_kaggle)

    df = pd.DataFrame(data=output_data_for_kaggle,columns=['id','target'])
    df.to_csv(file,index=False)

## SVM & Naive Bayes
Source : https://scikit-learn.org/stable/auto_examples/applications/plot_out_of_core_classification.html#sphx-glr-auto-examples-applications-plot-out-of-core-classification-py

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
import time
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import itertools

In [ ]:
train_input_text = train_inputs[:,1]
train_labels = train_labels.astype(int)

X_train_text, X_test_text, y_train, y_test = train_test_split(
    train_input_text, train_labels, test_size=0.2, random_state=0)

In [ ]:
# source : https://www.geeksforgeeks.org/ml-mini-batch-gradient-descent-with-python/
def create_mini_batches(X, y, batch_size):
    mini_batches = []
    data = np.hstack((X, y))
    np.random.shuffle(data)
    n_minibatches = data.shape[0] // batch_size
    i = 0
 
    for i in range(n_minibatches + 1):
        mini_batch = data[i * batch_size:(i + 1)*batch_size, :]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    if data.shape[0] % batch_size != 0:
        mini_batch = data[i * batch_size:data.shape[0]]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    return mini_batches

In [ ]:
vectorizer = HashingVectorizer(
    decode_error="ignore", n_features=2**27, alternate_sign=False, lowercase=True, binary=True
)

X_test = vectorizer.transform(X_test_text)

In [ ]:
class_weight = dict(enumerate(dist_train_labels, 0))

Naive_bayes_mode = True

if Naive_bayes_mode == True:
  cls = MultinomialNB(alpha=1,class_prior=dist_train_labels) #Naive Bayes
else:
  cls = SGDClassifier(max_iter=5, class_weight=class_weight, 
                    alpha=0.0001, penalty='l2', early_stopping=False, validation_fraction=0.1) #SVM

batch_mode = False

if batch_mode == True:
  batch_size = 1000
  i = 1
  mini_batches = create_mini_batches(np.array([X_train_text]).T, np.array([y_train]).T,batch_size)
  for mini_batch in mini_batches:
      i += 1
      X, y = mini_batch
      X_train = vectorizer.transform(X[:,0])
      cls.partial_fit(X_train, y[:,0].astype(int), classes=np.array([0,1,2]))
      if(i%10) == 0:
        print(i, X_train.shape, cls.score(X_test, y_test))
else:
  X_train = vectorizer.transform(X_train_text)
  cls.fit(X_train, y_train)

In [ ]:
cls.score(X_train, y_train)
#best fit SVM : 0.7734152149934275
#best fit NB : 0.8132682413386234

0.8132682413386234

In [ ]:
cls.score(X_test, y_test)
#best fit SVM : 0.7713695239468435
#best fit NB : 0.7789777233076202

0.7789777233076202

In [ ]:
#pour NB uniquement
pred_proba = cls.predict_proba(X_train)

indice_possible_neutre = np.abs(np.round(pred_proba[:,0]-0.5,2)).astype(float) <= 0.01
print(np.sum(indice_possible_neutre))
possible_neutre = y_train[indice_possible_neutre]
count_neg = len(possible_neutre[possible_neutre==0])
count_pos = len(possible_neutre[possible_neutre==2])
print(possible_neutre)
print(count_neg/len(y_train), count_pos/len(y_train))
y_train_neutral = np.copy(y_train)
y_train_neutral[indice_possible_neutre] = 1

dist_y_train_neutral = [np.sum(y_train_neutral==0)/len(y_train_neutral),
  np.sum(y_train_neutral==1)/len(y_train_neutral),
  np.sum(y_train_neutral==2)/len(y_train_neutral)]

print(dist_y_train_neutral)

cls = MultinomialNB(alpha=1,class_prior=dist_y_train_neutral) #Naive Bayes
X_train = vectorizer.transform(X_train_text)
cls.fit(X_train, y_train_neutral)

cls.score(X_test, y_test)

[2 2 2 ... 2 0 0]
0.01902414876156192 0.02836620374931812
[0.48044116127450864, 0.04746244553972446, 0.4720963931857669]


0.7691682887559176

In [ ]:
test_input_text = test_inputs[:,1]
X_test_kaggle = vectorizer.transform(test_input_text)

test_pred = cls.predict(X_test_kaggle)

PrepareKaggleFile(test_inputs=test_inputs, test_predictions=test_pred, file='tests_label_svm.csv')

[[     0      0]
 [     1      2]
 [     2      0]
 ...
 [560172      0]
 [560173      2]
 [560174      2]]


##Random prediction

In [ ]:
# random prediction but aligned with the prior distribution of training labels
p=dist_train_labels
test_pred_alea = np.random.choice(list_classes, len(test_inputs),p=p)

PrepareKaggleFile(test_inputs=test_inputs, test_predictions=test_pred_alea, file='tests_label_alea.csv')

##Neural Network with Bert Embeddings and preprocessing

References:


*   Bert Embeddings & Neural Network architechture: https://www.youtube.com/watch?v=hOCDJyZ6quA
https://www.youtube.com/watch?v=7kLi8u2dJz0
*   Data preprocessing techniques: https://www.mdpi.com/2078-2489/12/9/374/htm
*   Data augmentation: https://datascience.stackexchange.com/questions/27671/how-do-you-apply-smote-on-text-classification

(Will arrange the cells properly after submission)



In [ ]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.3 MB/s 
     |████████████████████████████████| 588.3 MB 5.3 kB/s 
     |████████████████████████████████| 439 kB 49.1 MB/s 
     |████████████████████████████████| 1.7 MB 42.4 MB/s 
     |████████████████████████████████| 6.0 MB 39.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.

In [ ]:
# Deep learning dependencies
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
# Data preprocessing dependencies
import nltk 
nltk.download('stopwords')

import re                                  
import string                             
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Conversion from categorical labels to numeric labels
df_train_result[df_train_result["target"]=='negative']=0
df_train_result[df_train_result["target"]=='neutral']=1
df_train_result[df_train_result["target"]=='positive']=2

In [ ]:
# Checking the train content
df_train.head()

,id,text
0,0,Anyway Im getting of for a while
1,1,"My red, Apache isn't feelin too well this morn..."
2,2,@danyelljoy you should be its great. friday w...
3,3,its 11:30pm and i dont wanna sleep; so i debat...
4,4,Why does twitter eat my DM's? Not happy


In [ ]:
# Check the test content
df_train_result.head()

,id,target
0,2,2
1,0,0
2,2,2
3,2,2
4,0,0


In [ ]:
# Creating a copy of dataset to point them to the different locations in the memory.
df_train_clean = df_train.copy()

In [ ]:
# Converting all the upper case to lower case to avoid the distinction between them
df_train_clean['text'] = df_train_clean.text.str.lower()
# Putting the regex for removing the https and www URLs
df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))

In [ ]:
# Remove the video and links
df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r'{link}', '', x))
df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r"\[video\]", '', x))

In [ ]:
# Remove html reference characters
df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r'&[a-z]+;', '', x))

In [ ]:
# Remove usernames
df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r'@[^\s]+', '', x))

In [ ]:
# Removing numbers
df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r'\d+', '', x))

In [ ]:
# Removing hashmarks, non-letter characters
df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))

In [ ]:
df_train_clean.head(20)

,id,text
0,0,anyway im getting of for a while
1,1,my red apache isn't feelin too well this morni...
2,2,you should be its great friday will be great...
3,3,its :pm and i dont wanna sleep; so i debated w...
4,4,why does twitter eat my dm's not happy
5,5,hey there drivin north i guess we will miss u...
6,6,is making cheese today in biology
7,7,cant sleep its already : am
8,8,what a rainy gloomy weekcant even get into our...
9,9,some bitch stole my blackberry the other night...


In [ ]:
# Remove punctuation and numbers
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df_train_clean['text'] = df_train_clean['text'].apply(lambda x: remove_punct(x))
df_train_clean.head(10)

,id,text
0,0,anyway im getting of for a while
1,1,my red apache isnt feelin too well this morning
2,2,you should be its great friday will be great...
3,3,its pm and i dont wanna sleep so i debated wit...
4,4,why does twitter eat my dms not happy
5,5,hey there drivin north i guess we will miss u...
6,6,is making cheese today in biology
7,7,cant sleep its already am
8,8,what a rainy gloomy weekcant even get into our...
9,9,some bitch stole my blackberry the other night...


In [ ]:
# Remove stop words
stopword_list = stopwords.words('english')
df_train_clean['text'] = df_train_clean['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopword_list)]))
df_train_clean.head(20)

,id,text
0,0,anyway im getting
1,1,red apache isnt feelin well morning
2,2,great friday great tooooooo
3,3,pm dont wanna sleep debated end decided perfec...
4,4,twitter eat dms happy
5,5,hey drivin north guess miss u tonite
6,6,making cheese today biology
7,7,cant sleep already
8,8,rainy gloomy weekcant even get new pool
9,9,bitch stole blackberry night santa monica stil...


In [ ]:
"""
Snowball Stemmer is also known as the Porter2 stemming algorithm 
because it is a better version of the Porter Stemmer. It is more aggressive than Porter Stemmer.
"""
from nltk.stem.snowball import SnowballStemmer
snowBallStemmer = SnowballStemmer("english")
def stemming(text):
    text = [snowBallStemmer.stem(word) for word in text.split()]
    return ' '.join(text)

df_train_clean['text'] = df_train_clean['text'].apply(lambda x: stemming(x))
df_train_clean.head()

,id,text
0,0,anyway im get
1,1,red apach isnt feelin well morn
2,2,great friday great tooooooo
3,3,pm dont wanna sleep debat end decid perfect ti...
4,4,twitter eat dms happi


In [ ]:
df_train_clean.head(20)

,id,text
0,0,anyway im get
1,1,red apach isnt feelin well morn
2,2,great friday great tooooooo
3,3,pm dont wanna sleep debat end decid perfect ti...
4,4,twitter eat dms happi
5,5,hey drivin north guess miss u tonit
6,6,make chees today biolog
7,7,cant sleep alreadi
8,8,raini gloomi weekcant even get new pool
9,9,bitch stole blackberri night santa monica stil...
